In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [2]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

In [3]:
import gradio as gr

# Initialize an empty list to maintain chat history & Message history
chat_history = []
msg_history = []
file_uploaded = False  # Flag to track if a file is uploaded

# Function to handle file upload
def handle_file_upload(file):
    global file_uploaded, chat_history, msg_history
    if file:
        # Reset histories on new file upload
        chat_history = []
        msg_history = []

        file_uploaded = True  # Set the flag to True when a file is uploaded

        # Extract the file name
        file_name = file.name

        # Add the "File Loaded" message to msg_history only
        msg_history.append({'role': "User", 'content': "File Loaded"})
        
        # Add the bot's response with the file name to both chat_history and msg_history
        bot_message = f"The file name is: {file_name}"
        chat_history.append(("Bot", bot_message))
        msg_history.append({'role': "assistant", 'content': bot_message})

        # Generate the formatted chat history
        conversation_history = format_chat_history(chat_history)

        return "File uploaded successfully. You can now start chatting!", conversation_history
    else:
        file_uploaded = False
        return "Please upload a valid Excel file.", ""

# Function to format chat history for display
def format_chat_history(chat_history):
    conversation_history = '''
        <div style="border: 2px solid #ddd; padding: 20px; max-height: 300px; overflow-y: auto; background: #f0f0f0; display: flex; flex-direction: column; border-radius: 10px; box-shadow: 0 4px 10px rgba(0,0,0,0.1);">
    '''
    for speaker, message in chat_history:
        if speaker == "User":
            conversation_history += f'''
                <div style="display: flex; justify-content: flex-start; margin-bottom: 10px;">
                    <div style="background-color: #d0f7d6; color: #3b8e3a; padding: 10px; border-radius: 12px; max-width: 60%; word-wrap: break-word;">
                        <b>{speaker}:</b> {message}
                    </div>
                </div>
            '''
        else:
            conversation_history += f'''
                <div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
                    <div style="background-color: #ffffff; padding: 10px; border-radius: 12px; max-width: 60%; word-wrap: break-word;">
                        <b>{speaker}:</b> {message}
                    </div>
                </div>
            '''
    conversation_history += "</div>"
    return conversation_history

# Function to enable chat input and submit button after file upload
def enable_chat_components(file_status):
    if "successfully" in file_status.lower():
        return gr.update(interactive=True), gr.update(interactive=True)
    else:
        return gr.update(interactive=False), gr.update(interactive=False)

# Function to handle chat history
def chatbot_history(user_input):
    global chat_history  # Use the global chat_history list
    global msg_history

    # Append user input to the chat history
    chat_history.append(("User", user_input))
    msg_history.append({'role': "User", 'content': user_input})

    # Generate a response (you can replace this with more complex logic)
    bot_response = f"Bot: {user_input[::-1]}"  # Reverse input for fun
    msg_history.append({'role': "assistant", 'content': bot_response})

    # Append bot response to chat history
    chat_history.append(("Bot", bot_response))

    # Create a formatted string of the conversation history
    conversation_history = format_chat_history(chat_history)

    print(msg_history)

    # Return the formatted conversation history and clear the input field
    return conversation_history, ""

# Create Gradio interface using Blocks with Soft theme
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Add a title with proper font
    gr.HTML("""
        <div style="font-size: 2.5em; color: #4CAF50; font-weight: bold; text-align: center;">
            Oh Sheet!!!!! It's Spark
        </div>
    """)

    # Layout with chat and input section
    with gr.Row():
        # Message section on the left (30% width)
        with gr.Column(scale=1, min_width=30):
            # File uploader at the top
            file_upload = gr.File(label="Upload an Excel File", file_types=[".xls", ".xlsx"], interactive=True)
            file_status = gr.Textbox(label="", interactive=False, lines=1, value="Please upload an Excel file to begin.", show_label=False)

            # Chatbox and submit button
            input_text = gr.Textbox(label="Enter your message", lines=1, interactive=False)  # Initially disabled
            submit_button = gr.Button("Submit", variant="primary", interactive=False)  # Initially disabled

        # Chatbot section on the right (70% width)
        with gr.Column(scale=2, min_width=70):
            output_text = gr.HTML(label="Chatbot History", elem_id="chat_history")

    # Ensure file upload enables the chatbox
    file_upload.change(fn=handle_file_upload, inputs=file_upload, outputs=[file_status, output_text])
    file_status.change(fn=enable_chat_components, inputs=file_status, outputs=[input_text, submit_button])

    # Ensure Enter key is mapped to submit functionality
    input_text.submit(fn=chatbot_history, inputs=input_text, outputs=[output_text, input_text])
    submit_button.click(fn=chatbot_history, inputs=input_text, outputs=[output_text, input_text])

# Launch the app
demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


[{'role': 'User', 'content': 'File Loaded'}, {'role': 'assistant', 'content': 'The file name is: C:\\Users\\Jasjyot Singh Jaswal\\AppData\\Local\\Temp\\gradio\\4632aaabe273965ebffeed34d2c5bf93291c6ff8ac33923f936e5fefae66fe43\\SalesData1.xlsx'}, {'role': 'User', 'content': 'Awesome dude'}, {'role': 'assistant', 'content': 'Bot: edud emosewA'}]
[{'role': 'User', 'content': 'File Loaded'}, {'role': 'assistant', 'content': 'The file name is: C:\\Users\\Jasjyot Singh Jaswal\\AppData\\Local\\Temp\\gradio\\4632aaabe273965ebffeed34d2c5bf93291c6ff8ac33923f936e5fefae66fe43\\SalesData1.xlsx'}, {'role': 'User', 'content': 'Awesome dude'}, {'role': 'assistant', 'content': 'Bot: edud emosewA'}, {'role': 'User', 'content': 'Welcome back'}, {'role': 'assistant', 'content': 'Bot: kcab emocleW'}]
